In [4]:
!pip install datacommons_pandas

   ---------------------------------------- 0.0/45.8 kB ? eta -:--:--
   ---------------------------------------- 45.8/45.8 kB ? eta 0:00:00


In [5]:
import datacommons_pandas as dc

In [6]:
city_dcids = dc.get_property_values(["CDC500_City"], "member", limit=500)["CDC500_City"]
city_dcids[:5]

['geoId/0107000',
 'geoId/0135896',
 'geoId/0137000',
 'geoId/0150000',
 'geoId/0151000']

In [7]:
data = dc.build_multivariate_dataframe(city_dcids,
                                       ["Percent_Person_Obesity", # Prevalence of obesity from CDC
                                        "Percent_Person_WithHighBloodPressure", # Prevalence of high blood pressure from CDC
                                        "UnemploymentRate_Person", # Unemployment rate from BLS
                                        "Count_Person_BelowPovertyLevelInThePast12Months", # Persons living below the poverty line from Census
                                        "Count_Person", # Total population from Census
                                       ]
                                      )
# Display the first five rows.
data.head(5)

,Percent_Person_Obesity,Percent_Person_WithHighBloodPressure,UnemploymentRate_Person,Count_Person_BelowPovertyLevelInThePast12Months,Count_Person
place,,,,,
geoId/0107000,44.9,45.6,3.3,49921,200431
geoId/0135896,32.5,33.0,2.0,4965,91995
geoId/0137000,43.7,37.9,2.4,28809,215025
geoId/0150000,41.3,40.6,3.7,37379,186316
geoId/0151000,37.9,40.7,2.9,40625,199819


In [8]:
dc.build_multivariate_dataframe(["country/USA", "geoId/1714000", "geoId/06085"],["CumulativeCount_MedicalConditionIncident_COVID_19_ConfirmedOrProbableCase"])

,CumulativeCount_MedicalConditionIncident_COVID_19_ConfirmedOrProbableCase
place,
country/USA,103910034
geoId/06085,342015


## COLLECTION THE STATISTICAL VALUES

In [39]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from webdriver_manager.chrome import ChromeDriverManager

# Set up the WebDriver (you can use any driver like ChromeDriver, GeckoDriver, etc.)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Define a function to recursively click elements with the class name 'title'
def recursively_click_titles(driver):
    # Using a set to keep track of clicked elements to avoid clicking the same element twice
    clicked_titles = set()

    def click_title_elements():
        # Find all elements with the class 'title'
        elements = driver.find_elements(By.CLASS_NAME, 'title')
        for element in elements:
            if element.tag_name !='span':
                continue
            # Check if the element has already been clicked to avoid re-clicking
            if element not in clicked_titles:
                try:
                    # Scroll into view and click the element
                    # ActionChains(driver).move_to_element(element).perform()
                    element.click()
                    
                    
                    # Add the element to the set of clicked titles
                    clicked_titles.add(element)
                    
                    # Wait for a moment to allow any dynamic content to load
                    # time.sleep(2)
                    
                    # Recursively call the function to handle new elements
                    click_title_elements()
                except Exception as e:
                    print(f"Error clicking element: {e}")
                    continue
    
    # Start the recursive clicking process
    click_title_elements()


# Navigate to the webpage
driver.get("https://datacommons.org/tools/statvar#s=dc%2Fs%2FUsFederalReserve")
driver.maximize_window()
# Wait for the page to fully load
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "title"))
)
recursively_click_titles(driver)

Error clicking element: Message: element click intercepted: Element <span class="title">...</span> is not clickable at point (148, 251). Other element would receive the click: <div id="tree-widget-tooltip" style="visibility: visible; left: 141px; top: 165px;">...</div>
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x00A9B8E3+45827]
	(No symbol) [0x00A2DCC4]
	(No symbol) [0x0092150F]
	(No symbol) [0x00968052]
	(No symbol) [0x009663D8]
	(No symbol) [0x0096425B]
	(No symbol) [0x00963823]
	(No symbol) [0x009585EF]
	(No symbol) [0x00982DFC]
	(No symbol) [0x00958075]
	(No symbol) [0x00983094]
	(No symbol) [0x0099C034]
	(No symbol) [0x00982B96]
	(No symbol) [0x00956998]
	(No symbol) [0x0095751D]
	GetHandleVerifier [0x00D54513+2899763]
	GetHandleVerifier [0x00DA793D+3240797]
	GetHandleVerifier [0x00B213B4+593364]
	GetHandleVerifier [0x00B282DC+621820]
	(No symbol) [0x00A370A4]
	(No symbol) [0x00A337A8]
	(No symbol) [0x00A33947]
	(No symbol) [0x00A259FE]
	BaseThreadIni

In [ ]:
elems = driver.find_elements(By.CLASS_NAME,value="node-title")